In [1]:
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
from __future__ import division
import pickle
import random

### Get list of movies

Find 300 top grossing movies per year for 2004 - 2014

In [2]:
url_base = 'http://www.imdb.com/search/title?at=0&sort=boxoffice_gross_us&start=' 
url_feat = '&title_type=feature&year='

In [136]:
titles = {}
counts = [i for i in range(1,300,50)]

In [137]:
year = 2005

for count in counts:
    start = count
    url = url_base + str(start) + url_feat + str(year)
    print "Getting data " + str(start) + " for " + str(year)
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page)

    lst = soup.find_all(attrs={"href" : re.compile("/title/")})
    for link in lst:
        if len(link.text) > 0 and link.text != 'X':
            titles[link.attrs['href']] = link.text

Getting data 1 for 2005
Getting data 51 for 2005
Getting data 101 for 2005
Getting data 151 for 2005
Getting data 201 for 2005
Getting data 251 for 2005


In [138]:
sorted(titles.values())[:10]

[u'12 and Holding',
 u'13 Tzameti',
 u'A History of Violence',
 u'A Lot Like Love',
 u'A Sound of Thunder',
 u'Adam & Steve',
 u'Alone in the Dark',
 u'An American Haunting',
 u'An Unfinished Life',
 u'Angel-A']

In [139]:
len(titles.items())

300

In [140]:
len(movies_dct.items())

2644

In [60]:
# movies_dct = {}

In [71]:
def add_movie_to_data(soup2, href):
    title = soup2.find(itemprop = "name").text
    metacritic = str(soup2.find(attrs={"href" : re.compile("criticreviews?")}).text.strip())
    info = soup2.find(class_ = 'infobar')
    meta = info.find_all('meta')
    
    genres = []
    genres2 = info.find_all(itemprop='genre')
    for genre in genres2:
        genres.append(str(genre.text))
    duration = info.find(itemprop='duration').attrs['datetime']
    opening = soup2.find(text=re.compile("Opening Weekend:")).next.strip()
    studio = soup2.find(attrs={"href" : re.compile("/company/")}).text.strip()
    
    try:
        rating = info.find(itemprop='contentRating')['content']
    except TypeError:
        rating = 'UNKNOWN'
    
    try:
        release = info.find(itemprop='datePublished')['content']
    except TypeError:
        release = 0
    
    # get correct gross
    url_bus = url_page + 'business?ref_=tt_dt_bus'
    response3 = requests.get(url_bus)
    page3 = response3.text
    soup3 = BeautifulSoup(page3)
    
    try:
        gross = soup3.find(text=re.compile("Gross")).next.strip().split()[0]
    except AttributeError:
        gross = soup2.find(text=re.compile("Gross:")).next.strip()
    
    movies_dct[href] = {'title': title,
                         'metacritic': metacritic,
                         'rating': rating,
                         'genres': genres,
                         'release': release,
                         'duration': duration,
                         'studio': studio,
                         'gross': gross,
                         'opening': opening
                        }
    if len(movies_dct.items()) % 50 == 0:
        with open('my_data_partial.pkl', 'w') as picklefile:
            pickle.dump(movies_dct.items(), picklefile)
            print "dumped " + str(len(movies_dct.items()))
    return movies_dct

In [133]:
for href, title in titles.items():
    url_page = 'http://www.imdb.com/' + href

    try:
        response2 = requests.get(url_page)
        page2 = response2.text
        soup2 = BeautifulSoup(page2)
        add_movie_to_data(soup2, href)
        sleep(0.8)

    except AttributeError:
        print title + ' did not load'
        sleep(0.7)

dumped 2350
Believe in Me did not load
dumped 2400
The Work and the Glory III: A House Divided did not load
Fanaa did not load
Humko Deewana Kar Gaye did not load
dumped 2450
dumped 2500
dumped 2550
dumped 2600
Shaadi Karke Phas Gaya Yaar did not load


In [134]:
len(movies_dct.items())

2644

In [135]:
(len(movies_dct.items())-2349)/len(titles.items())

0.9833333333333333

In [124]:
movies_dct.items()[:4]

[('/title/tt1433905/',
  {'duration': 'PT144M',
   'genres': ['Action', 'Comedy', 'Drama'],
   'gross': u'$96,054',
   'metacritic': 'Metacritic Reviews',
   'opening': u'$96,054        \n\n      (USA)',
   'rating': 'UNKNOWN',
   'release': '2009-10-16',
   'studio': u'Cinema Capital',
   'title': u'All the Best: Fun Begins'}),
 ('/title/tt2191701/',
  {'duration': 'PT101M',
   'genres': ['Comedy'],
   'gross': u'$133,471,453',
   'metacritic': '19/100',
   'opening': u'$41,508,572        \n\n      (USA)',
   'rating': 'PG-13',
   'release': '2013-07-12',
   'studio': u'Columbia Pictures',
   'title': u'Grown Ups 2'}),
 ('/title/tt1735898/',
  {'duration': 'PT127M',
   'genres': ['Action', 'Adventure', 'Drama'],
   'gross': u'$155,136,755',
   'metacritic': '57/100',
   'opening': u'$56,217,700        \n\n      (USA)',
   'rating': 'PG-13',
   'release': '2012-06-01',
   'studio': u'Roth Films',
   'title': u'Snow White and the Huntsman'}),
 ('/title/tt2737050/',
  {'duration': 'PT95M

In [159]:
with open('my_data_2000_2014.pkl', 'w') as picklefile:
    pickle.dump(movies_dct.items(), picklefile)

In [168]:
count = 0
for i in range(len(movies_dct.items())):
    if (movies_dct.items()[i][1]['metacritic'] == 'Metacritic Reviews'):
        count += 1
        
print count

316


In [169]:
count / len(movies_dct.items())

0.07172038129822969

### MISC JUNK

In [44]:
url_page = 'http://www.imdb.com/title/tt3093546/'

response2 = requests.get(url_page)
page2 = response2.text
soup2 = BeautifulSoup(page2)

In [ ]:
#soup2.prettify()

In [45]:
soup2.find(itemprop = "name").text

u'Listen Up Philip'

In [46]:
str(soup2.find(attrs={"href" : re.compile("criticreviews?")}).text.strip())

'76/100'

In [47]:
info = soup2.find(class_ = 'infobar')

In [48]:
len(info)

13

In [49]:
info.find(itemprop='contentRating')['content']

TypeError: 'NoneType' object has no attribute '__getitem__'

In [50]:
meta = info.find_all('meta')

In [51]:
meta

[<meta content="2014-10-21" itemprop="datePublished"/>]

In [52]:
info.find(itemprop='duration').attrs['datetime']

'PT108M'

In [53]:
str(soup2.find(text=re.compile("Opening Weekend:")).next.strip())

'$23,284        \n\n      (USA)'

In [54]:
str(soup2.find(attrs={"href" : re.compile("/company/")}).text.strip())

'Sailor Bear'

In [55]:
str(soup2.find(text=re.compile("Release Date:")).next.strip())

'21 October 2014 (USA)'

In [56]:
url_bus = url_page + 'business?ref_=tt_dt_bus'
response3 = requests.get(url_bus)
page3 = response3.text
soup3 = BeautifulSoup(page3)

In [57]:
url_bus

'http://www.imdb.com/title/tt3093546/business?ref_=tt_dt_bus'

In [58]:
soup3.find(text=re.compile("Gross")).next.strip().split()[0]

AttributeError: 'NoneType' object has no attribute 'next'